### Fitxer de prova

In [ ]:
/usr/bin/python3 -m pip install ipykernel -U --user --force-reinstall

: 

In [ ]:
# Importar librerías necesarias
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Cargar el dataset principal de solicitudes de adelanto en efectivo
cash_request = pd.read_csv('../data/extract - cash request - data analyst.csv')

# Inspeccionar la estructura y el tipo de datos de 'cash_request'
cash_request.info()

: 

In [ ]:
# Crear una copia del DataFrame para preservar el original
cr = cash_request.copy()
# Filtrar columnas relevantes para el análisis
cr = cr[["amount", "created_at", "user_id", "deleted_account_id","transfer_type"]]
cr.info()


In [ ]:
# Convertir 'created_at' a tipo datetime para manipulación de fechas
cr['created_at'] = pd.to_datetime(cr['created_at'])
cr.head()


In [ ]:
# Convertir la columna 'created_at' a una fecha sin zona horaria
cr['created_at'] = cr['created_at'].dt.tz_localize(None)
#cr['created_at_notz'] = cr['created_at'].dt.tz_localize(None)
cr.head()

In [ ]:
cr['Mes'] = cr['created_at'].dt.to_period('M')
cr.head()


In [ ]:
# Crear columna 'Cohorte' que indica el mes de la primera solicitud de cada cliente
cr['Cohorte'] = cr.groupby('user_id')['created_at'].transform('min').dt.to_period('M')
#cr['Cohorte'] = cr.groupby('user_id')['created_at_notz'].transform('min').dt.to_period('M')
cr.head(10)

In [ ]:
# Calcular el número inicial de clientes únicos en cada cohorte
cohort_sizes_initial = cr.groupby('Cohorte').agg(Num_Clientes_Iniciales=('user_id', 'nunique')).reset_index()
cohort_sizes_initial


In [ ]:
# Calcular la cantidad de clientes únicos en cada cohorte para cada mes
cohort_sizes_monthly = cr.groupby(['Cohorte', 'Mes']).agg(Num_Clientes=('user_id', 'nunique')).reset_index()
cohort_sizes_monthly

In [ ]:
# Calcular la cantidad de solicitudes de adelanto de efectivo de los clientes de cada cohorte en cada mes
cohort_cash_monthly = cr.groupby(['Cohorte', 'Mes']).agg(Num_Solicitudes=('user_id', 'count')).reset_index()
cohort_cash_monthly

In [ ]:
# Calcular la tasa de retención de clientes (clientes únicos por cohorte en cada mes / clientes iniciales de la cohorte)
retention_clientes = cohort_sizes_monthly.pivot(index='Cohorte', columns='Mes', values='Num_Clientes')
retention_clientes = retention_clientes.div(cohort_sizes_initial.set_index('Cohorte')['Num_Clientes_Iniciales'], axis=0)
retention_clientes

In [ ]:
# Visualizar la tasa de retención de clientes usando un mapa de calor
plt.figure(figsize=(10, 8))
plt.title('Tasa de Retención de Clientes por Cohorte')
sns.heatmap(data=retention_clientes, annot=True, fmt='.0%', vmin=0.0, vmax=1.0, cmap='BuGn', cbar_kws={'label': 'Tasa de Retención'})
plt.xlabel('Meses desde la primera solicitud de adelanto en efectivo')
plt.ylabel('Cohorte')
plt.show()


In [ ]:
# Calcular la tasa de abandono de clientes (1 - tasa de retención)
churn_clientes = 1 - retention_clientes


In [ ]:

plt.figure(figsize=(10, 8))
plt.title('Tasa de Abandono de Clientes por Cohorte')
sns.heatmap(data=churn_clientes, annot=True, fmt='.0%', vmin=0.0, vmax=1.0,cmap='OrRd', cbar_kws={'label': 'Tasa de Abandono'})
plt.xlabel('Meses desde la primera solicitud de adelanto en efectivo')
plt.ylabel('Cohorte')
plt.show()


In [ ]:
# Calcular el número inicial de solicitudes de adelanto de efectivo en cada cohorte
cohort_cash_initial = cohort_cash_monthly.groupby('Cohorte').first().reset_index().rename(columns={'Num_Solicitudes':'Num_Solicitudes_Iniciales'})
cohort_cash_initial


In [ ]:
# Calcular la tasa de retención de solicitudes (solicitudes por cohorte en cada mes / solicitudes iniciales de la cohorte)
retention_cash = cohort_cash_monthly.pivot(index='Cohorte', columns='Mes', values='Num_Solicitudes')
retention_cash = retention_cash.div(cohort_cash_initial.set_index('Cohorte')['Num_Solicitudes_Iniciales'], axis=0)
retention_cash

In [ ]:
# Visualizar la tasa de retención de solicitudes usando un mapa de calor
plt.figure(figsize=(10, 8))
plt.title('Tasa de Retención de Solicitudes de Adelanto de Efectivo por Cohorte')
sns.heatmap(data=retention_cash, annot=True, fmt='.0%', vmin=0.0, vmax=1.0, cmap='Blues', cbar_kws={'label': 'Tasa de Retención'})
plt.xlabel('Meses desde la primera solicitud de adelanto en efectivo')
plt.ylabel('Cohorte')
plt.show()

In [ ]:
# Calcular la tasa de abandono de solicitudes (1 - tasa de retención de solicitudes)
churn_cash = 1 - retention_cash
churn_cash

In [ ]:
# Visualizar la tasa de abandono de solicitudes usando un mapa de calor
plt.figure(figsize=(10, 8))
plt.title('Tasa de Abandono de Solicitudes de Adelanto de Efectivo por Cohorte')
sns.heatmap(data=churn_cash, annot=True, fmt='.0%', vmin=0.0, vmax=1.0, cmap='Oranges', cbar_kws={'label': 'Tasa de Abandono'})
plt.xlabel('Meses desde la primera solicitud de adelanto en efectivo')
plt.ylabel('Cohorte')
plt.show()

In [ ]:
# Cargar el dataset de tarifas o fees para análisis de cohortes de tarifas
fees = pd.read_csv('../data/extract - fees - data analyst - .csv')
fees.info()

In [ ]:
# Crear una copia del DataFrame para preservar el original
fs = fees.copy()
fs = fs[["cash_request_id", "total_amount", "created_at"]]
fs.info()

In [ ]:
# Convertir la columna 'created_at' en fees a tipo datetime para análisis temporal
fs['created_at'] = pd.to_datetime(fs['created_at'])
fs.head()

In [ ]:
# Convertir la columna 'created_at' a una fecha sin zona horaria
fs['created_at'] = fs['created_at'].dt.tz_localize(None)
fs.head()

In [ ]:
# Crear columna 'Mes' indicando el mes de cada solicitud
fs['Mes'] = fs['created_at'].dt.to_period('M')
fs.head()

In [ ]:
# Crear columna 'Cohorte' que indica el mes del primer cargo de cada cliente
fs['Cohorte'] = fs.groupby('cash_request_id')['created_at'].transform('min').dt.to_period('M')
fs.head()

In [ ]:
# Calcular la cantidad de tarifas iniciales por cohorte
cohort_fees_initial = fs.groupby('Cohorte').agg(Num_Tarifas_Iniciales=('cash_request_id', 'nunique')).reset_index()
cohort_fees_initial

In [ ]:
# Calcular el número de tarifas en cada cohorte para cada mes
cohort_fees_monthly = fs.groupby(['Cohorte', 'Mes']).agg(Num_Tarifas=('cash_request_id', 'nunique')).reset_index()
cohort_fees_monthly

In [ ]:
# Calcular la tasa de retención de tarifas (número de tarifas de cada cohorte en cada mes / tarifas iniciales de la cohorte)
retention_fees = cohort_fees_monthly.pivot(index='Cohorte', columns='Mes',values='Num_Tarifas')
retention_fees = retention_fees.div(cohort_fees_initial.set_index('Cohorte')['Num_Tarifas_Iniciales'], axis=0)
retention_fees

In [ ]:
# Visualizar la tasa de retención de tarifas usando un mapa de calor
plt.figure(figsize=(10, 8))
plt.title('Tasa de Retención de Tarifas por Cohorte')
sns.heatmap(data=retention_fees, annot=True, fmt='.0%', vmin=0.0, vmax=1.0,cmap='Greens', cbar_kws={'label': 'Tasa de Retención'})
plt.xlabel('Meses desde la primera tarifa')
plt.ylabel('Cohorte')
plt.show()


In [ ]:
churn_fees = 1 - retention_fees

plt.figure(figsize=(10, 8))
plt.title('Tasa de Abandono de Tarifas por Cohorte')
sns.heatmap(data=churn_fees, annot=True, fmt='.0%', vmin=0.0, vmax=1.0,cmap='Reds', cbar_kws={'label': 'Tasa de Abandono'})
plt.xlabel('Meses desde la primera tarifa')
plt.ylabel('Cohorte')
plt.show()

In [ ]:
# Agregar columnas de temporalidad
# Semana y mes de la solicitud
cr['Semana_Mes'] = cr['created_at'].dt.strftime('%U_%B') + '_' + cr['Mes'].dt.strftime('%Y')
# Día y semana de la solicitud
cr['Dia_Semana'] = cr['created_at'].dt.strftime('%A') + '_' + cr['Semana_Mes']
# Hora y dia de la solicitud
cr['Hora_Dia'] = cr['created_at'].dt.hour.astype(str) + '_' + cr['Dia_Semana']
cr.head()

In [ ]:
# Calcular la cantidad de adelanto de efectivo por cohorte y mes
cohort_revenue = cr.groupby(['Cohorte', 'Mes'])['amount'].sum().reset_index()
cohort_revenue = cohort_revenue.pivot(index='Cohorte', columns='Mes', values='amount')
cohort_revenue

In [ ]:
# Graficar la cantidad de adelanto de efectivo por cohorte y mes
plt.figure(figsize=(10, 8))
plt.title('Cantidad de Adelanto de Efectivo por Cohorte y Mes')
sns.heatmap(data=cohort_revenue, annot=True, fmt='.0f', cmap='viridis', annot_kws={"size": 8}, cbar_kws={'label': 'Cantidad de Adelanto de Efectivo'})
plt.xlabel('Mes por Año')
plt.ylabel('Cohorte')
plt.show()


In [ ]:
# Crear DataFrame de cohortes para el mes de '2020-03'
cohort_amount_2020_03 = cr[cr['Cohorte'] == '2020-03'].groupby('user_id')['amount'].sum().reset_index()
cohort_amount_2020_03

In [ ]:
# Ordenar por el monto total para encontrar los 10 usuarios principales
sort_cohort_amount_2020_03 = cohort_amount_2020_03.sort_values(by='amount', ascending=False)
top_10_users = sort_cohort_amount_2020_03.head(10)
top_10_users

In [ ]:
# Filtrar para incluir solo los 10 usuarios seleccionados
amount_2020_03 = cr[cr['Cohorte'] == '2020-03'].groupby(['user_id','Mes'])['amount'].sum().reset_index()
cohort_amount_2020_03_top10 = amount_2020_03[amount_2020_03['user_id'].isin(top_10_users['user_id'])]
cohort_amount_2020_03_top10 = cohort_amount_2020_03_top10.pivot(index='user_id', columns='Mes', values='amount')
cohort_amount_2020_03_top10

In [ ]:
# Graficar la cantidad de adelanto de efectivo de un cohorte por meses y número de usuarios
plt.figure(figsize=(10, 8))
plt.title('Cantidad de Adelanto de Efectivo del Cohorte 2020-03 por Meses por Año y Número de Usuario')
sns.heatmap(data=cohort_amount_2020_03_top10, annot=True, fmt='.0f', cmap='viridis', annot_kws={"size": 8}, cbar_kws={'label': 'Cantidad de Adelanto de Efectivo'})
plt.xlabel('Mes por Año')
plt.ylabel('Número de Usuario')
plt.show()


In [ ]:
# Filtrar para calcular la cantidad de adelanto de efectivo por cohorte y semana
cohort_weekly_amount = cr[cr['Cohorte'] == '2020-03'].groupby(['user_id','Semana_Mes'])['amount'].sum().reset_index()
cohort_weekly_amount

In [ ]:
# Filtrar para incluir solo los 10 usuarios seleccionados
cohort_weekly_amount_top10 = cohort_weekly_amount[cohort_weekly_amount['user_id'].isin(top_10_users['user_id'])]
cohort_weekly_amount_top10 = cohort_weekly_amount_top10.pivot(index='user_id', columns='Semana_Mes', values='amount')
cohort_weekly_amount_top10

In [ ]:
# Seleccionar las primeras 10 semanas
cohort_weekly_amount_top10 = cohort_weekly_amount_top10.loc[:,cohort_weekly_amount_top10.columns[:10]]
cohort_weekly_amount_top10

In [ ]:
# Graficar la cantidad de adelanto de efectivo de los 10 principales usuarios por semana
plt.figure(figsize=(10, 8))
plt.title('Cantidad de Adelanto de Efectivo del Cohorte 2020-03 por Semana por Mes y Número de Usuario')
sns.heatmap(data=cohort_weekly_amount_top10, annot=True, fmt='.0f', cmap='viridis', annot_kws={"size": 8}, cbar_kws={'label': 'Cantidad de Adelanto de Efectivo'})
plt.xlabel('Semana por Mes')
plt.ylabel('Número de Usuario')
plt.show()



In [ ]:
# Filtrar para calcular la cantidad de adelanto de efectivo por cohorte y día de la semana
cohort_daily_amount = cr[ (cr['Cohorte'] == '2020-03') & (cr['Semana_Mes'] == '10_March_2020') ].groupby(['user_id', 'Dia_Semana'])['amount'].sum().reset_index()
cohort_daily_amount

In [ ]:
# Filtrar para incluir solo los 10 usuarios seleccionados
cohort_daily_amount_top10 = cohort_daily_amount[cohort_daily_amount['user_id'].isin(top_10_users['user_id'])]
cohort_daily_amount_top10 = cohort_daily_amount_top10.pivot(index='user_id',columns='Dia_Semana', values='amount')
cohort_daily_amount_top10

In [ ]:
# Graficar la cantidad de adelanto de efectivo de los 10 principales usuarios por día de la semana
plt.figure(figsize=(10, 8))
plt.title('Cantidad de Adelanto de Efectivo del Cohorte 2020-03 por Día por Semana y Número de Usuario')
sns.heatmap(data=cohort_daily_amount_top10, annot=True, fmt='.0f', cmap='viridis', annot_kws={"size": 8}, cbar_kws={'label': 'Cantidad de Adelanto de Efectivo'})
plt.xlabel('Día por Semana')
plt.ylabel('Número de Usuario')
plt.show()



In [ ]:
# Filtrar para calcular la cantidad de adelanto de efectivo por cohorte y hora
cohort_hourly_amount = cr[(cr['Cohorte'] == '2020-03') & 
                          (cr['Semana_Mes'] == '10_March_2020') & 
                          (cr['Dia_Semana'] == 'Monday_10_March_2020')].groupby(['user_id', 'Hora_Dia'])['amount'].sum().reset_index()
cohort_hourly_amount

In [ ]:
# Filtrar para incluir solo los 10 usuarios seleccionados
cohort_hourly_amount_top10 = cohort_hourly_amount[cohort_hourly_amount['user_id'].isin(top_10_users['user_id'])]
cohort_hourly_amount_top10 = cohort_hourly_amount_top10.pivot(index='user_id',columns='Hora_Dia', values='amount')
cohort_hourly_amount_top10

In [ ]:
# Graficar la cantidad de adelanto de efectivo de los 10 principales usuarios por hora
plt.figure(figsize=(10, 8))
plt.title('Cantidad de Adelanto de Efectivo del Cohorte 2020-03 por Hora por Dia y Número de Usuario')
sns.heatmap(data=cohort_hourly_amount_top10, annot=True, fmt='.0f', cmap='viridis', annot_kws={"size": 8}, cbar_kws={'label': 'Cantidad de Adelanto de Efectivo'})
plt.xlabel('Hora por Dia')
plt.ylabel('Número de Usuario')
plt.show()

In [ ]:
# Importando la biblioteca pandas para manipulación y análisis de datos
import pandas as pd
# Importando NumPy para operaciones numéricas y manipulación de arreglos
import numpy as np
# Importando Seaborn para visualización de datos estadísticos (opcional, pero puede complementar a Plotly)
import seaborn as sns
# Importando Plotly Express para visualizaciones interactivas de alto nivel y fáciles de usar
import plotly.express as px
# Importando matplotlib.pyplot para crear gráficos y visualizaciones

import matplotlib.pyplot as plt
# Importando Plotly Graph Objects para un control más detallado sobre las visualizaciones
import plotly.graph_objects as go
# Importando itertools para generar combinaciones de columnas
import itertools
# Importando la función seasonal_decompose para la descomposición de series temporales
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
cash_request=pd.read_csv('../data/extract - cash request - data analyst.csv')
fees=pd.read_csv('../data/extract - fees - data analyst - .csv')

fig1 = px.imshow(cash_request.isnull(),
color_continuous_scale=['black', 'red'],
labels=dict(color="NaN"),
title="Mapa de calor para cash_request")
fig1.update_layout(width=700, height=700)
fig1.update_coloraxes(showscale=False)
fig1.show()


In [ ]:
fig2 = px.imshow(fees.isnull(),
color_continuous_scale=['black', 'red'],
labels=dict(color="NaN"),
title="Mapa de calor para fees")
fig2.update_layout(width=700, height=700)
fig2.update_coloraxes(showscale=False)
fig2.show()


In [ ]:
cash_request.describe()

In [ ]:
fees.describe()

In [ ]:
cash_request.info()

In [ ]:
fees.info()

In [ ]:
#Revisión de columnas con valores clave
columnas_cash_request=cash_request.columns.tolist()
for i in columnas_cash_request:
    print("===========================")
    print(f"{cash_request[i].value_counts().head(10)}")

In [ ]:
columnas_fees=fees.columns.tolist()
for i in columnas_fees:
    print("===========================")
    print(f"{fees[i].value_counts().head(10)}")

In [ ]:
# Visualización de distribuciones de datos
columnas_numericas_cash_request=cash_request.describe().columns.tolist()
for c in columnas_numericas_cash_request:
    fig = px.histogram(cash_request, x=c, title=f'Histograma de {c}')
    fig.update_layout(height=500)
    fig.show()


In [ ]:
columnas_numericas_fees=fees.describe().columns.tolist()
for c in columnas_numericas_fees:
    fig = px.histogram(fees, x=c, title=f'Histograma de {c}')
    fig.update_layout(height=500)
    fig.show()


In [ ]:
for c in columnas_cash_request:
    if c not in columnas_numericas_cash_request:
        counts = cash_request[c].value_counts()
        if len(counts)<=10:
            fig = px.pie(names=counts.index, values=counts.values, title=f'Distribución de {c}')
            fig.update_layout(height=500)
            fig.show()

In [ ]:
for c in columnas_fees:
    if c not in columnas_numericas_fees:
        counts = fees[c].value_counts()
        if len(counts)<=10:
            fig = px.pie(names=counts.index, values=counts.values, title=f'Distribución de {c}')
            fig.update_layout(height=500)
            fig.show()



In [ ]:
for c in columnas_numericas_cash_request:
    fig = px.box(cash_request, y=c, title=f'Boxplot de {c}')
    fig.update_layout(height=500)
    fig.show()


In [ ]:
for c in columnas_numericas_fees:
    fig = px.box(fees, y=c, title=f'Boxplot de {c}')
    fig.update_layout(height=500)
    fig.show()


In [ ]:
for c in columnas_numericas_cash_request:
    fig = px.violin(cash_request, y=c, title=f'Violinplot de {c}')
    fig.update_layout(height=500)
    fig.show()


In [ ]:
for c in columnas_numericas_fees:
    fig = px.violin(fees, y=c, title=f'Violinplot de {c}')
    fig.update_layout(height=500)
    fig.show()

In [ ]:
# Genera gráficos de dispersión para todas las combinaciones de columnas numéricas
for c1, c2 in itertools.combinations(columnas_numericas_cash_request, 2):
    fig = px.scatter(cash_request, x=c1, y=c2, title=f'Gráfico de dispersión de {c1} vs {c2}')
    fig.update_layout(height=500)
    fig.show()



In [ ]:
# Genera gráficos de dispersión con línea de tendencia para todas las combinaciones de columnas numéricas
for c1, c2 in itertools.combinations(columnas_numericas_cash_request, 2):
    fig = px.scatter(cash_request, x=c1, y=c2,
    title=f'Gráfico de dispersión de {c1} vs {c2} con línea de tendencia',
    trendline="ols") # Agregar línea de tendencia (OLS)
    # Cambiar el color de la línea de tendencia a rojo
    fig.update_traces(line=dict(color='red', width=3)) # Color y ancho de la línea de tendencia
    fig.update_traces(line=dict(color='red'), selector=dict(mode='lines')) #Cambiar el color de la línea de tendencia
    fig.update_layout(height=500)
    fig.show()

In [ ]:
# Genera gráficos de dispersión con línea de tendencia para todas las combinaciones de columnas numéricas
for c1, c2 in itertools.combinations(columnas_numericas_fees, 2):
    fig = px.scatter(fees, x=c1, y=c2,

    title=f'Gráfico de dispersión de {c1} vs {c2} con línea de tendencia',
    trendline="ols") # Agregar línea de tendencia (OLS)
    # Cambiar el color de la línea de tendencia a rojo
    fig.update_traces(line=dict(color='red', width=3)) # Color y ancho de la línea de tendencia
    fig.update_traces(line=dict(color='red'), selector=dict(mode='lines')) #Cambiar el color de la línea de tendencia
    fig.update_layout(height=500)
    fig.show()

In [ ]:
#cash_request=pd.read_csv('../data/extract - cash request - data analyst.csv')
#cash_request.info()
cash_request.describe()
#cr_cp = cash_request.copy()
#cr_num=cr_cp.describe().columns#.tolist()
#print(cr_num)

In [ ]:
# Descomposición de series temporales para cada columna numérica individual
for c in columnas_numericas_cash_request:
    # Crear una copia del DataFrame para trabajar
    cash_request_copy = cash_request[c].copy()
    
    # Verifica si hay valores faltantes y elimínalos o interpola
    if cash_request_copy.isnull().any():
        cash_request_copy = cash_request_copy.interpolate() # Rellenar NaN con interpolación
    
    # Eliminar filas con valores faltantes
    cash_request_copy = cash_request_copy.dropna()
    # Verificar que no hay valores no finitos
    if not np.isfinite(cash_request_copy).all():
        print(f'La columna {c} contiene valores no finitos, se omite la descomposición.')
        continue
    # Descomposición de la serie temporal
    descomposicion = seasonal_decompose(cash_request_copy, model='additive', period=12)
    # Graficar la descomposición
    fig = descomposicion.plot()
    # Ajustar el tamaño de la figura con matplotlib
    46
    plt.gcf().set_size_inches(10, 6)
    # Ajustar el título para que no se superponga
    fig.suptitle(f'Descomposición de la serie temporal de {c}', fontsize=16, y=1.05)
    plt.show()

In [ ]:
# Descomposición de series temporales para cada columna numérica individual
for c in columnas_numericas_fees:
    # Crear una copia del DataFrame para trabajar
    fees_copy = fees[c].copy()
    # Verifica si hay valores faltantes y elimínalos o interpola
    if fees_copy.isnull().any():
        fees_copy = fees_copy.interpolate() # Rellenar NaN con interpolación
    
    # Eliminar filas con valores faltantes
    fees_copy = fees_copy.dropna()
    # Verificar que no hay valores no finitos
    if not np.isfinite(fees_copy).all():
        print(f'La columna {c} contiene valores no finitos, se omite la descomposición.')
        continue
    # Descomposición de la serie temporal
    descomposicion = seasonal_decompose(fees_copy, model='additive', period=12)
    # Graficar la descomposición
    fig = descomposicion.plot()
    49
    # Ajustar el tamaño de la figura con matplotlib
    plt.gcf().set_size_inches(10, 6) #
    # Ajustar el título para que no se superponga
    fig.suptitle(f'Descomposición de la serie temporal de {c}', fontsize=16, y=1.05)
    plt.show()